In [10]:
import PyPDF2

def fill_pdf_fields(pdf_path, output_path, field_data):
    # Open the input PDF file
    with open(pdf_path, 'rb') as input_pdf_file:
        pdf_reader = PyPDF2.PdfReader(input_pdf_file)
        pdf_writer = PyPDF2.PdfWriter()

        # Iterate through each page in the PDF
        for page in pdf_reader.pages:
            print('in for loop')
            # Check if the page has form fields
            if '/Annots' in page:
                for annot in page['/Annots']:
                    # Extract the field name
                    field_name = annot.get_object()['/T']

                    # If the field name exists in the field data, update the field value
                    if field_name in field_data:
                        print(field_name)
                        annot.update({
                            NameObject("/V"): TextStringObject(field_data[field_name])
                        })

            # Add the page to the PDF writer
            pdf_writer.add_page(page)

        # Write the updated PDF to the output file
        with open(output_path, 'wb') as output_pdf_file:
            pdf_writer.write(output_pdf_file)

    print(f"Filled PDF saved to {output_path}")

# Example usage
if __name__ == "__main__":
    parsed_data = {}

    field_data = {
        "SUSPECT ADVERSE REACTION REPORT": "Adverse Reaction Report by raja sekhar",
    }

    pdf_path = "C:/Users/sekhar/Downloads/test.pdf"
    output_path = "C:/Users/sekhar/Downloads/test2.pdf"

    fill_pdf_fields(pdf_path, output_path, field_data)
    print('Completed')


in for loop
Filled PDF saved to C:/Users/sekhar/Downloads/test2.pdf
Completed


Text added to PDF and saved to C:/Users/sekhar/Downloads/test2.pdf


In [19]:
import fitz

def fill_pdf_fields(pdf_path, output_path, field_data):
    pdf_document = fitz.open(pdf_path)
    
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        
        # Iterate over the fillable fields
        widgets = page.widgets()
        if widgets:
            for widget in widgets:
                if widget.field_type:  # Check if it's a fillable field
                    field_name = widget.field_name
                    if field_name in field_data:  # Check if the field name is in field_data
                        field_value = field_data[field_name]  # Get the value from field_data
                        widget.field_value = field_value  # Update the field value
                        widget.update()  # Apply the update to the widget
        
        # Search for specific text instances and replace them with corresponding values
        for key, value in field_data.items():
            text_instances = page.search_for(key)
            for text_instance in text_instances:
                page.insert_text(text_instance[:2], value)
    
    pdf_document.save(output_path)
    pdf_document.close()

    print(f"PDF saved to {output_path}")

# Example usage
if __name__ == "__main__":
    pdf_path = "C:/Users/sekhar/Downloads/Fillable-Form_CIOMS-to-E2B.pdf"
    output_path = "C:/Users/sekhar/Downloads/test2.pdf"

    field_data = {
        "SUSPECT ADVERSE REACTION REPORT": "Adverse Reaction Report by raja sekhar",
        "1a COUNTRY": 'INDIA',
    }
    
    fill_pdf_fields(pdf_path, output_path, field_data)


PDF saved to C:/Users/sekhar/Downloads/test2.pdf


In [8]:
import fitz

def add_text_to_pdf(pdf_path, output_path, text_to_add, position=(100, 150), font_size=12):
    # Open the PDF
    pdf_document = fitz.open(pdf_path)

    # Get the first page
    page = pdf_document[0]

    # Add text to the page
    page.insert_text(position, text_to_add, fontname="helv", fontsize=font_size)

    # Save the modified PDF
    pdf_document.save(output_path)
    pdf_document.close()

    print(f"Text added to PDF and saved to {output_path}")

# Example usage
if __name__ == "__main__":
    pdf_path = "C:/Users/sekhar/Downloads/Fillable-Form_CIOMS-to-E2B.pdf"
    output_path = "C:/Users/sekhar/Downloads/test2.pdf"
    text_to_add = "This is the text to add to the PDF."

    add_text_to_pdf(pdf_path, output_path, text_to_add)


Text added to PDF and saved to C:/Users/sekhar/Downloads/test2.pdf


In [46]:
import fitz

def fill_pdf_fields(pdf_path, output_path, field_data):
    pdf_document = fitz.open(pdf_path)
    
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        
        # Iterate over the fillable fields
        widgets = page.widgets()
        if widgets:
            for widget in widgets:
                if widget.field_type:  # Check if it's a fillable field
                    field_name = widget.field_name
                    print(widget.field_name)
                    if field_name in field_data:  # Check if the field name is in field_data
                        if widget.field_type != 2:
                            field_value = field_data[field_name]  # Get the value from field_data
                            widget.field_value = field_value  # Update the field value
                            widget.update()  # Apply the update to the widget
                            print('*****',widget.field_type)
                        elif widget.field_type == 2:
                            print('checkbox')
                            if field_value.lower() in ["yes", "true", "1"]:
                                widget.field_value = "1"  # Check the checkbox
                            else:
                                widget.field_value = "0"  # Uncheck the checkbox
                        widget.update()  # Apply the update to the widget
        
    
    pdf_document.save(output_path)
    pdf_document.close()

    print(f"PDF saved to {output_path}")

# Example usage
if __name__ == "__main__":
    pdf_path = "C:/Users/sekhar/Downloads/Fillable-Form_CIOMS-to-E2B.pdf"
    output_path = "C:/Users/sekhar/Downloads/test2.pdf"

    field_data = {
        "SUSPECT ADVERSE REACTION REPORT": "Adverse Reaction Report by raja sekhar",
        "1a COUNTRY": 'INDIA',
        "7  13 DESCRIBE REACTIONS including relevant testslab data": "Patient experienced severe headache and nausea.",
        "PatientDied": "yes",
        "3_Sex": "Male",
        "InpatientHospitalisation": "no",
        "Persistence_Incapacity": "yes",
        "LifeThreatening": "No",
        "14_SuspectDrugs": "Drug A",
        "15_DailyDoses": "2 times a day",
        "16_RoutesOfAdmin": "Oral",
    }
    
    fill_pdf_fields(pdf_path, output_path, field_data)


SUSPECT ADVERSE REACTION REPORT
***** 7
1 PATIENT INITIALS first last
1a COUNTRY
***** 7
2_DateOfBirth
2a AGE Years
3_Sex
***** 3
4_ReactionOnset
7  13 DESCRIBE REACTIONS including relevant testslab data
***** 7
PatientDied
checkbox
InpatientHospitalisation
checkbox
Persistence_Incapacity
checkbox
LifeThreatening
checkbox
14_SuspectDrugs
***** 7
20
20
20
15_DailyDoses
***** 7
16_RoutesOfAdmin
***** 7
17_IndicationsForUse
21
21
21
18_TherapyFrom
18_TherapyTo
19_TherapyDuration
22 CONCOMITANT DRUGS AND DATES OF ADMINISTRATION exclude those used to treat reaction
23 OTHER RELEVANT HISTORY eg diagnostics allergies pregnancy with last month of period etc
24A_NameOfManufacturer
24a  ADDRESS OF MANUFACTURER
24b MFR CONTROL NO
24_ManufacturerDetails
24c DATE RECEIVED BY MANUFACTURER
DATE OF THIS REPORT
24d_Study
24d_HealthProfessional
24d_Literature
25
25
PDF saved to C:/Users/sekhar/Downloads/test2.pdf


In [2]:
import pymupdf
types = set()
def fill_pdf_fields(pdf_path, output_path, field_data):
    pdf_document = pymupdf.open(pdf_path)
    
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        
        # Iterate over the fillable fields
        widgets = page.widgets()
        if widgets:
            for widget in widgets:
                types.add(widget.field_type)
                if widget.field_type:  # Check if it's a fillable field
                    field_name = widget.field_name
                    print(widget.field_type, '***********************' ,widget.field_name)
                    if field_name in field_data:  # Check if the field name is in field_data
                        field_value = field_data[field_name]  # Get the value from field_data
                        if widget.field_type != 2:
                            widget.field_value = field_value  # Update the field value
                            print(widget.field_type, )
                        else:
                            # Checkbox handling
                            # print('*')
                            # print(widget.field_value)
                            # print('*')
                            if field_value.lower() in ["yes", "true", "1"]:
                                widget.field_value = widget.on_state()  # Check the checkbox
                            else:
                                widget.field_value = '0'  # Uncheck the checkbox
                        # print(widget.field_name, widget.field_value)
                        widget.update()  # Apply the update to the widget
                        
    
    pdf_document.save(output_path)
    pdf_document.close()
    print(types)
    print(f"PDF saved to {output_path}")

# Example usage
if __name__ == "__main__":
    pdf_path = "C:/Users/sekhar/Downloads/Fillable-Form_CIOMS-to-E2B.pdf"
    output_path = "C:/Users/sekhar/Downloads/test2.pdf"

    field_data = {
        "SUSPECT ADVERSE REACTION REPORT": "Adverse Reaction Report of Mushroom Solutions",
        "1a COUNTRY": 'INDIA',
        "7  13 DESCRIBE REACTIONS including relevant testslab data": "Patient experienced severe headache and nausea.",
        "PatientDied": "yes",
        "3_Sex": "Male",
        "InpatientHospitalisation": "yes",
        "Persistence_Incapacity": "yes",
        "LifeThreatening": "No",
        "14_SuspectDrugs": "Drug A",
        "15_DailyDoses": "2 times a day",
        "16_RoutesOfAdmin": "Oral",
    }
    
    fill_pdf_fields(pdf_path, output_path, field_data)


7 *********************** SUSPECT ADVERSE REACTION REPORT
7
7 *********************** 1 PATIENT INITIALS first last
7 *********************** 1a COUNTRY
7
7 *********************** 2_DateOfBirth
7 *********************** 2a AGE Years
3 *********************** 3_Sex
3
7 *********************** 4_ReactionOnset
7 *********************** 7  13 DESCRIBE REACTIONS including relevant testslab data
7
2 *********************** PatientDied
2 *********************** InpatientHospitalisation
2 *********************** Persistence_Incapacity
2 *********************** LifeThreatening
7 *********************** 14_SuspectDrugs
7
5 *********************** 20
5 *********************** 20
5 *********************** 20
7 *********************** 15_DailyDoses
7
7 *********************** 16_RoutesOfAdmin
7
7 *********************** 17_IndicationsForUse
5 *********************** 21
5 *********************** 21
5 *********************** 21
7 *********************** 18_TherapyFrom
7 *********************** 18_Th